다차원 벡터를 나타내는 클래스를 생성해서 앞 장에서 구현한 2차원 `Vector2d` 클래스를 개선하여 다음과 같은 기능을 지원하게 하려고 한다.

* 기본 시퀀스 프로토콜 : `__len__()`과 `__getitem__()` 메서드
* 여러 항목을 가진 객체를 안전하게 표현
* 슬라이싱을 지원해서 새로운 벡터 객체 생성
* 포함된 요소 값을 모두 고려한 집합 해싱
* 커스터마이즈된 포맷 언어 확장
* `__getattr__()` 메서드로 동적 속성 접근 구현

In [1]:
!cat vector2d_v3.py

# vector2d_v3.py

from array import array
import math

class Vector2d:
    typecode = 'd' # Vector2d와 bytes 간의 변환에 사용하는 클래스 속성
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y) # 미리 실수로 변환하는 센스
        
    @property # 게터 메서드를 나타냄
    def x(self):
        return self.__x        
        
    @property
    def y(self):
        return self.__y

    def __hash__(self):
        return hash(self.x) ^ hash(self.y)        
    
    def __iter__(self): # 이걸 구현하면 x,y = my_vector 처럼 쓸 수 있다.
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)])+
               bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        re

# 10.1 Vector: 사용자 정의 시퀀스형

벡터의 요소들을 실수형 배열에 저장하고, 벡터가 불변 균일 시퀀스처럼 작동하게 만들기 위해 필요한 메서드를 구현한다

# 10.2 Vector 버전 #1: Vector2d 호환

최초의 `Vector` 버전은 앞에서 구현한 `Vector2d` 클래스와 가능한 호환성이 높도록 구현한다. 다만 시퀀스 생성자는 내장 시퀀스처럼 반복형을 인수로 받도록 한다.

In [2]:
from array import array
import reprlib
import math

class Vector:
    typecode = 'd'
    
    def __init__(self, components):
        # 벡터 요소를 배열로 저장
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components) # 반복할 수 있도록 구현
    
    def __repr__(self):
        components = reprlib.repr(self._components) # 제한된 길이로 출력
        components = components[components.find('['):-1] # 문자열 중 앞에 나오는 "array('d'," 를 제거
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
               bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv) # 언패킹할 필요가 없음

`reprlib.repr()` 을 사용해서 내부에 배열을 사용한다는 내용을 감추었다. 

# 10.3 프로토콜과 덕 타이핑

프로토콜은 문서에만 정의되어 있고 실제 코드에서는 정의되지 않는 비공식 인터페이스이다. 예를 들어 파이썬에서 시퀀스형을 만들기 위해서는 어떤 클래스를 상속할 필요 없이 시퀀스 프로토콜(`__len__()`과 `__getitem__()`)에 따르는 메서드를 구현하면 된다. 가령 예제 1-1을 보자.

In [3]:
import collections

Card = collections.namedtuple('Card', ['rank','suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                      for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

`FrenchDeck` 클래스는 시퀀스 프로토콜을 구현하므로 파이썬에서 제공하는 여러 기능을 활용할 수 있다. 이 클래스가 시퀀스처럼 동작하기 때문에 시퀀스이다. 이 메커니즘을 __덕 타이핑__이라고 한다.

클래스가 사용되는 환경에 따라 프로토콜의 일부만 구현할 수도 있다. 예를 들어 반복을 지원하려면 `__getitem__()` 메서드만 구현하면 된다.

# 10.4 Vector 버전 #2: 슬라이스 가능한 시퀀스

`Vector` 클래스 안에 시퀀스 프로토콜을 구현하자. 

In [5]:
from array import array
import reprlib
import math

class Vector:
    typecode = 'd'
    
    def __init__(self, components):
        # 벡터 요소를 배열로 저장
        self._components = array(self.typecode, components)
    
    """시퀀스 프로토콜 구현"""
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        return self._components[index]
    """시퀀스 프로토콜 구현 종료"""
    
    def __iter__(self):
        return iter(self._components) # 반복할 수 있도록 구현
    
    def __repr__(self):
        components = reprlib.repr(self._components) # 제한된 길이로 출력
        components = components[components.find('['):-1] # 문자열 중 앞에 나오는 "array('d'," 를 제거
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
               bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv) # 언패킹할 필요가 없음

이 두 메서드가 추가되었으니 다음과 같은 연산 수행이 가능하다.

In [6]:
v1 = Vector([3,4,5]) 
v1

Vector([3.0, 4.0, 5.0])

In [7]:
len(v1), v1[0], v1[-1]

(3, 3.0, 5.0)

In [9]:
v7 = Vector(range(7))
v7, v7[1:4]

(Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...]), array('d', [1.0, 2.0, 3.0]))

보다시피 슬라이싱도 지원되긴 하나 객체가 배열로 바뀌게 된다. 따라서 슬라이싱해서 `Vector` 객체를 생성하려면 슬라이싱 연산을 배열에 위임하면 안되며, `__getitem__()` 메서드에 구현해야 한다.

## 10.4.1 슬라이싱의 작동 방식

실제 슬라이싱이 어떻게 작동하는지 확인해보자.

In [10]:
class MySeq:
    def __getitem__(self, index):
        return index

In [11]:
s = MySeq()
s[1]

1

In [12]:
s[1:4]

slice(1, 4, None)

In [13]:
s[1:4:2]

slice(1, 4, 2)

In [17]:
s[1:4:2, 9] # [] 안에 콤마가 들어가면 __getitem__() 이 튜플을 받는다.

(slice(1, 4, 2), 9)

In [16]:
s[1:4:2, 7:9]

(slice(1, 4, 2), slice(7, 9, None))

`slice`에 대해 살펴보자.

In [19]:
dir(slice) 

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'indices',
 'start',
 'step',
 'stop']

`start`, `stop`, `step`과 `indices()` 메서드를 볼 수 있다.

In [20]:
help(slice.indices)

Help on method_descriptor:

indices(...)
    S.indices(len) -> (start, stop, stride)
    
    Assuming a sequence of length len, calculate the start and stop
    indices, and the stride length of the extended slice described by
    S. Out of bounds indices are clipped in a manner consistent with the
    handling of normal slices.



즉, 길이가 `l`인 리스트에 슬라이스를 적용한 결과값을 반환한다.

In [23]:
st = "ABCDE"

In [25]:
slice(None, 10, 2).indices(5)

(0, 5, 2)

In [30]:
st[:10:2], st[:10:2] == st[0:5:2]

('ACE', True)

In [22]:
slice(-3, None, None).indices(5)

(2, 5, 1)

In [31]:
st[-3::], st[-3::] == st[2:5:1]

('CDE', True)

실제 `Vector` 구현에서는 `slice` 인수를 받을 때 `_components` 배열에 처리를 위임할 것이므로 `slice.indices()` 메서드를 구현할 필요가 없지만, 기반 시퀀스가 제공하는 서비스에 의존할 수 없을 때는 이 메서드가 도움이 된다.

## 10.4.2 슬라이스를 인식하는 `__getitem__()`

이제는 `__getitem__()`이 슬라이싱도 제대로 처리하도록 구현해보자. 먼저 `type()` 메서드를 확인해보자.

In [36]:
cls = type(v1) # v1의 원래 클래스를 반환
cls([1,2])

Vector([1.0, 2.0])

In [43]:
# vector_v2.py

from array import array
import reprlib
import math
import numbers

class Vector:
    typecode = 'd'
    
    def __init__(self, components):
        # 벡터 요소를 배열로 저장
        self._components = array(self.typecode, components)
    
    """시퀀스 프로토콜 구현"""
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self) # 객체의 클래스를 가져옴
        if isinstance(index, slice): # index가 슬라이스이면
            return cls(self._components[index]) # Vector 객체를 생성
        elif isinstance(index, numbers.Integral): # index가 정수형이면
            return self._components[index] # 해당 항목 반환
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls)) 
    """시퀀스 프로토콜 구현 종료"""
    
    def __iter__(self):
        return iter(self._components) # 반복할 수 있도록 구현
    
    def __repr__(self):
        components = reprlib.repr(self._components) # 제한된 길이로 출력
        components = components[components.find('['):-1] # 문자열 중 앞에 나오는 "array('d'," 를 제거
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
               bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv) # 언패킹할 필요가 없음

In [52]:
v7 = Vector(range(7))
v7[-1] # 정수형 인덱스는 단 한 요소의 값을 실수형으로 반환

6.0

In [53]:
v7[1:4] # 슬라이스 인덱스는 Vector를 새로 만든다

Vector([1.0, 2.0, 3.0])

In [54]:
v7[-1:] # 길이가 1인 슬라이스도 Vector 객체를 생성한다

Vector([6.0])

In [55]:
v7[1,2] # 다차원 인덱싱은 지원하지 않으므로 에러 발생

TypeError: Vector indices must be integers

# 10.5 Vector 버전 #3: 동적 속성 접근

`Vector` 객체는 `v.x`, `v.y`처럼 벡터 요소를 이름으로 접근하는 기능이 없으므로 이를 구현해보자. `Vector2d`는 `@property` 데커레이터를 이요하여 구현하였으나 조금 귀찮았다... 따라서 특정 속성을 찾지 못하면 인터프리터가 호출하는 `__getattr__()` 메서드를 이용하여 깔끔하게 구현해보자.

In [56]:
# vector_v3.py

from array import array
import reprlib
import math
import numbers

class Vector:
    typecode = 'd'
    shortcut_names = 'xyzt'
    
    def __init__(self, components):
        # 벡터 요소를 배열로 저장
        self._components = array(self.typecode, components)
    
    """시퀀스 프로토콜 구현"""
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self) # 객체의 클래스를 가져옴
        if isinstance(index, slice): # index가 슬라이스이면
            return cls(self._components[index]) # Vector 객체를 생성
        elif isinstance(index, numbers.Integral): # index가 정수형이면
            return self._components[index] # 해당 항목 반환
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls)) 
    """시퀀스 프로토콜 구현 종료"""
    
    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1: # name이 한글자이면 
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components): # 포지션이 범위 내에 있으면 배열 항목 반환
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}' 
        raise AttributeError(msg.format(cls, name))
    
    def __iter__(self):
        return iter(self._components) # 반복할 수 있도록 구현
    
    def __repr__(self):
        components = reprlib.repr(self._components) # 제한된 길이로 출력
        components = components[components.find('['):-1] # 문자열 중 앞에 나오는 "array('d'," 를 제거
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
               bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv) # 언패킹할 필요가 없음

In [65]:
v = Vector(range(5))
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

In [66]:
dir(v)

['__abs__',
 '__bool__',
 '__bytes__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_components',
 'frombytes',
 'shortcut_names',
 'typecode']

In [59]:
v.x, v.y

(0.0, 1.0)

그러나 다음과 같은 오류가 발생한다.

In [68]:
v.x = 10
v.x, v # 벡터 요소는 변경되지 않았음

(10, Vector([0.0, 1.0, 2.0, 3.0, 4.0]))

이는 `v.x = 10` 을 선언하는 순간 `v` 객체에 `x` 속성이 추가되므로, 더 이상 `v.x`를 호출할 때 `__getattr()`을 호출하지 않기 때문이다.

In [69]:
dir(v)[-1] # x 라는 속성이 생김

'x'

이를 해결하기 위해서 소문자 하나로 된 속성명에 값을 할당할 때 예외를 발생시키려고 한다. 따라서 다음과 같이 `__setattr__()` 메서드를 구현하자.

In [70]:
# vector_v3.py

from array import array
import reprlib
import math
import numbers

class Vector:
    typecode = 'd'
    shortcut_names = 'xyzt'
    
    def __init__(self, components):
        # 벡터 요소를 배열로 저장
        self._components = array(self.typecode, components)
    
    """시퀀스 프로토콜 구현"""
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self) # 객체의 클래스를 가져옴
        if isinstance(index, slice): # index가 슬라이스이면
            return cls(self._components[index]) # Vector 객체를 생성
        elif isinstance(index, numbers.Integral): # index가 정수형이면
            return self._components[index] # 해당 항목 반환
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls)) 
    """시퀀스 프로토콜 구현 종료"""
    
    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1: # name이 한글자이면 
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components): # 포지션이 범위 내에 있으면 배열 항목 반환
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}' 
        raise AttributeError(msg.format(cls, name))
        
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_names:
                error = 'readonly attribute {attr_name!r}' # xyzt 중 하나는 구체적으로 오류 발생
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}" # 그외 소문자면 일반적 메세지 오류 발생
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        # 에러가 발생하지 않을 때는 정상적으로 __setattr__() 메서드 호출
        super().__setattr__(name, value) 
        
                
    
    def __iter__(self):
        return iter(self._components) # 반복할 수 있도록 구현
    
    def __repr__(self):
        components = reprlib.repr(self._components) # 제한된 길이로 출력
        components = components[components.find('['):-1] # 문자열 중 앞에 나오는 "array('d'," 를 제거
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
               bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv) # 언패킹할 필요가 없음

In [71]:
v = Vector(range(5))
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

In [72]:
v.x

0.0

In [73]:
v.x = 10

AttributeError: readonly attribute 'x'

In [74]:
v.u = 10

AttributeError: can't set attributes 'a' to 'z' in 'Vector'

In [78]:
v.test = 10
dir(v) # 이러면 적용 가능

['__abs__',
 '__bool__',
 '__bytes__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_components',
 'frombytes',
 'shortcut_names',
 'test',
 'typecode']

따라서 객체 동작의 불일치를 피하려면 `__getattr__()`을 구현할 때 `__setattr__()`도 함께 구현해야 한다. 또한 벡터 요소의 변경을 허용하고 싶은 경우, `__setitem__()` 메서드를 구현하면 `v[0]=1.1`의 형태로, `__setattr__()` 메서드를 구현하면 `v.x = 1.1`로 작성할 수 있다.

# 10.6 Vector 버전 #4: 해싱 및 더 빠른 ==

`__hash__()` 메서드를 구현해보자. 이제는 여러개의 요소를 반복적용해야 하므로 `functools.reduce()` 함수를 이용해야 한다.

0에서 5까지의 숫자에 XOR을 누적 계산하는 세 가지 방법을 소개한다.

In [79]:
n = 0
for i in range(6):
    n ^= i
n

1

In [80]:
import functools
functools.reduce(lambda a,b:a^b, range(6))

1

In [81]:
import operator
functools.reduce(operator.xor, range(6))

1

`functools`와 `operator` 모듈을 임포트해서 `Vector.__hash__()`를 구현하자.

In [83]:
# vector_v4.py

from array import array
import reprlib
import math
import numbers
import functools
import operator

class Vector:
    typecode = 'd'
    shortcut_names = 'xyzt'
    
    def __init__(self, components):
        # 벡터 요소를 배열로 저장
        self._components = array(self.typecode, components)
    
    """시퀀스 프로토콜 구현"""
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self) # 객체의 클래스를 가져옴
        if isinstance(index, slice): # index가 슬라이스이면
            return cls(self._components[index]) # Vector 객체를 생성
        elif isinstance(index, numbers.Integral): # index가 정수형이면
            return self._components[index] # 해당 항목 반환
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls)) 
    """시퀀스 프로토콜 구현 종료"""
    
    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1: # name이 한글자이면 
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components): # 포지션이 범위 내에 있으면 배열 항목 반환
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}' 
        raise AttributeError(msg.format(cls, name))
        
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_names:
                error = 'readonly attribute {attr_name!r}' # xyzt 중 하나는 구체적으로 오류 발생
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}" # 그외 소문자면 일반적 메세지 오류 발생
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        # 에러가 발생하지 않을 때는 정상적으로 __setattr__() 메서드 호출
        super().__setattr__(name, value) 
        
                
    
    def __iter__(self):
        return iter(self._components) # 반복할 수 있도록 구현
    
    def __repr__(self):
        components = reprlib.repr(self._components) # 제한된 길이로 출력
        components = components[components.find('['):-1] # 문자열 중 앞에 나오는 "array('d'," 를 제거
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
               bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __hash__(self):
        hashes = (hash(x) for x in self._components) # 제너레이터 표현식 이용
        return functools.reduce(operator.xor, hasehs, 0) # 초기값을 0으로 함
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv) # 언패킹할 필요가 없음

`map` 함수를 쓰면 다음처럼 구현할 수도 있다.

```python
def __hash__(self):
    hashes = map(hash, self._components)
    return functools.reduce(operator.xor, hashes)
```

또한 현재 `__eq__()` 구현은 단지 비교를 위해 튜플 두개를 만들어야 하므로 비효율적이다. 따라서 다음과 같이 구현하는 것이 낫다.

In [85]:
# vector_v4.py

from array import array
import reprlib
import math
import numbers
import functools
import operator

class Vector:
    typecode = 'd'
    shortcut_names = 'xyzt'
    
    def __init__(self, components):
        # 벡터 요소를 배열로 저장
        self._components = array(self.typecode, components)
    
    """시퀀스 프로토콜 구현"""
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self) # 객체의 클래스를 가져옴
        if isinstance(index, slice): # index가 슬라이스이면
            return cls(self._components[index]) # Vector 객체를 생성
        elif isinstance(index, numbers.Integral): # index가 정수형이면
            return self._components[index] # 해당 항목 반환
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls)) 
    """시퀀스 프로토콜 구현 종료"""
    
    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1: # name이 한글자이면 
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components): # 포지션이 범위 내에 있으면 배열 항목 반환
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}' 
        raise AttributeError(msg.format(cls, name))
        
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_names:
                error = 'readonly attribute {attr_name!r}' # xyzt 중 하나는 구체적으로 오류 발생
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}" # 그외 소문자면 일반적 메세지 오류 발생
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        # 에러가 발생하지 않을 때는 정상적으로 __setattr__() 메서드 호출
        super().__setattr__(name, value) 
        
                
    
    def __iter__(self):
        return iter(self._components) # 반복할 수 있도록 구현
    
    def __repr__(self):
        components = reprlib.repr(self._components) # 제한된 길이로 출력
        components = components[components.find('['):-1] # 문자열 중 앞에 나오는 "array('d'," 를 제거
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
               bytes(self._components))
    
    def __eq__(self, other):
        if len(self) != len(other): # 길이가 다르면 False
            return False
        for a, b in zip(self, other): # 제너레이터로부터 하나씩 비교
            if a != b:
                return False
        return True
    
    def __hash__(self):
        hashes = (hash(x) for x in self._components) # 제너레이터 표현식 이용
        return functools.reduce(operator.xor, hasehs, 0) # 초기값을 0으로 함
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv) # 언패킹할 필요가 없음

이 부분을 `all()` 함수를 이용하여 다음과 같이 구현할 수도 있다.

```python
def __eq__(self, other):
    return len(self) == len(other) and all(a==b fora,b in zip(self, other)
```

`zip()`은 가장 짧은 피연산자에서 멈추므로 먼저 길이를 검사하는 것이 좋다.

# 10.7 Vector 버전 #5: 포매팅

`Vector`의 `__format__()` 메서드는 극좌표 대신 구면좌표를 이용한다. 이를 위해서는 `angle(n)` 메서드를 통해 특정 좌표의 각좌표를 계산하고, `angles()`를 이용하여 모든 각좌표의 반복형을 반환한다.

<img src='https://wikimedia.org/api/rest_v1/media/math/render/svg/9259a6efe54e3274fa6dbb10433d54eb84387c85' style="background-color:white;padding:20px;">

<img src='https://wikimedia.org/api/rest_v1/media/math/render/svg/8b2848ec784176c56b02f21bcefe05829570e507' style="background-color:white;padding:20px;">

<img src='https://wikimedia.org/api/rest_v1/media/math/render/svg/c46d0b1117225d3c47b6b855edef72f48e405c60' style="background-color:white;padding:20px;">

...

<img src='https://wikimedia.org/api/rest_v1/media/math/render/svg/6183f6d990adc7c99146a16971cea907e06619a8' style="background-color:white;padding:20px;">

이를 다음과 같이 구현하자.

```python
def angle(self, n):
    r = math.sqrt(sum(x * x for x in self[n:]))
    a = math.atan2(r, self[n-1])
    if (n == len(self) - 1) and (self[-1]<0):
        return math.pi * 2 - a
    else:
        return a

def angles(self):
    return (self.angle(n) for n in range(1, len(self)))

def __format__(self, fmt_spec=''):
    if fmt_spec.endswith('h'): # 초구면좌표
        fmt_spec = fmt_spec[:-1]
        coords = itertools.chain([abs(self)],
                                self.angles())
        outer_fmt = '<{}>'
    else:
        coords = self
        outer_fmt = '({})'
    components = (format(c, fmt_spec) for c in coords)
    return outer_fmt.format(', '.join(components))

```

여기서 `itertools.chain()` 함수는 여러개의 시퀀스를 입력으로 받았을 때 이를 하나의 연결된 시퀀스처럼 처리한다.

In [90]:
a = [1,2,3]
b =[4,5,6]
for i in itertools.chain(a,b):
    print(i)

1
2
3
4
5
6


In [93]:
# vector_v5.py

from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools # chain() 함수를 사용하기 위함

class Vector:
    typecode = 'd'
    shortcut_names = 'xyzt'
    
    def __init__(self, components):
        # 벡터 요소를 배열로 저장
        self._components = array(self.typecode, components)
    
    """시퀀스 프로토콜 구현"""
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self) # 객체의 클래스를 가져옴
        if isinstance(index, slice): # index가 슬라이스이면
            return cls(self._components[index]) # Vector 객체를 생성
        elif isinstance(index, numbers.Integral): # index가 정수형이면
            return self._components[index] # 해당 항목 반환
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls)) 
    """시퀀스 프로토콜 구현 종료"""
    
    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1: # name이 한글자이면 
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components): # 포지션이 범위 내에 있으면 배열 항목 반환
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}' 
        raise AttributeError(msg.format(cls, name))
        
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_names:
                error = 'readonly attribute {attr_name!r}' # xyzt 중 하나는 구체적으로 오류 발생
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}" # 그외 소문자면 일반적 메세지 오류 발생
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        # 에러가 발생하지 않을 때는 정상적으로 __setattr__() 메서드 호출
        super().__setattr__(name, value) 
        
    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1]<0):
            return math.pi * 2 - a
        else:
            return a
        
    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'): # 초구면좌표
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                    self.angles()) 
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))
    
    def __iter__(self):
        return iter(self._components) # 반복할 수 있도록 구현
    
    def __repr__(self):
        components = reprlib.repr(self._components) # 제한된 길이로 출력
        components = components[components.find('['):-1] # 문자열 중 앞에 나오는 "array('d'," 를 제거
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
               bytes(self._components))
    
    def __eq__(self, other):
        if len(self) != len(other): # 길이가 다르면 False
            return False
        for a, b in zip(self, other): # 제너레이터로부터 하나씩 비교
            if a != b:
                return False
        return True
    
    def __hash__(self):
        hashes = (hash(x) for x in self._components) # 제너레이터 표현식 이용
        return functools.reduce(operator.xor, hasehs, 0) # 초기값을 0으로 함
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv) # 언패킹할 필요가 없음

In [104]:
v1 = Vector([1,2,3])
v1

Vector([1.0, 2.0, 3.0])

In [105]:
format(v1, '0.3fh')

'<3.742, 1.300, 0.983>'